In [92]:
import nltk   
from bs4 import BeautifulSoup
from collections import Counter
import magic
import sys 
import time
from nltk.corpus import stopwords
import re
from os import listdir
from os.path import isfile, join
import numpy as np
import heapq
from collections import defaultdict
from pymystem3 import Mystem
from multiprocessing import Pool
import pickle

In [93]:
patt = re.compile (r'[\W^\d]+', flags=re.UNICODE)

In [94]:
stemm = Mystem (entire_input=True)

In [95]:
stemm.start ()

In [96]:
def read_urls (f_name):
    urls = {}
    with open(f_name) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    for url in content:
        split = url.split ('\t')
        urls[split[1].decode ('utf-8')] = int(split[0])
    return urls

In [97]:
def read_queries (f_name):
    urls = {}
    patt = re.compile (r'[\W^\d]+', flags=re.UNICODE)
    with open(f_name) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    for url in content:
        split = url.split ('\t')
        urls[int(split[0])] = set ([word for word in set (patt.split (split[1].decode ('utf-8').lower ())) if word not in stopwords.words('russian')])
        if u'' in urls[int(split[0])]:
            urls[int(split[0])].remove (u'')
    return urls

In [98]:
class doc_reader:
    def __init__ (self, path, stemm, urls):
        with open(path) as f:
            self.doc_num = urls[f.readline ().rstrip()]
            html = f.read()
            m = magic.Magic(mime_encoding=True)
            encoding = m.from_buffer(html)
            try:
                html = html.decode (encoding)
            except:
    #             sys.stderr.write ("{}, unknown encoding. skipping".format (doc_name))
                return None
            
        soup = BeautifulSoup(html)

        for script in soup(["script", "style"]):
            script.decompose()    # rip it out
        self.body = self.clear_text (soup.find ('body').get_text ().lower (), stemm)
        self.title = self.clear_text (soup.find ('title').get_text ().lower (), stemm)
        pass

    def clear_text (self, text, stemm):
        patt = re.compile (r'[\w]+', flags=re.UNICODE)
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk for chunk in chunks if chunk)
        return [word for word in stemm.lemmatize (text) if patt.search(word)]

In [99]:
class docs_container:
    def __init__ (self, path, stemm, urls):
        self.path = path
        self.stemm = stemm
        self.urls = urls
        self.files = [path + f for f in listdir (path)]
        self.docs = {}
        
    def read_docs (self):
        for f_name in self.files:
            doc = doc_reader (f_name, self.stemm, self.urls)
            if doc is None:
                continue
            self.docs[doc.doc_num] = doc

In [100]:
def scan_dir (self, dir_name):
    d_c = docs_container (dir_name)
    d_c.read_docs ()
    pickle.dump (d_c, dir_name + "scanned.dump")

class doc_reader:
    def __init__ (self, path, stemm, urls):
        self.path = path
        self.stemm = stemm
        self.urls = urls
        self.dirs = listdir (path)
        self.n_pools = len (self.dirs)
        
    def run (self):
        for d in self.dirs:
            with Pool(8) as p:
                p.map(scan_dir, d)

In [101]:
reader.n_pools

8

In [102]:
reader = doc_reader ("content/", stemm, urls)

In [103]:
reader.run ()

AttributeError: __exit__

In [12]:
urls = read_urls ("urls.numerate.txt")

In [6]:
queries = read_queries ("queries.numerate.txt")

In [7]:
def read_doc (doc_name):
    with open(doc_name) as f:
        name = f.readline ()
        html = f.read()
        m = magic.Magic(mime_encoding=True)
        encoding = m.from_buffer(html)
        try:
            html = html.decode (encoding)
        except:
#             sys.stderr.write ("{}, unknown encoding. skipping".format (doc_name))
            return None
            
    soup = BeautifulSoup(html)

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return name, text.lower ()

In [8]:
def count_idf (queries):
    mypath = 'content/'
    dirs = listdir(mypath)
    onlyfiles = []
    len_of_file = []
    for d in dirs:
        onlyfiles.extend ([mypath + d + '/' + f for f in listdir(mypath + d) ])
        print onlyfiles
    docs_for_querie = defaultdict (set)
    t = time.time ()
    words = Counter ()
    num = len (onlyfiles)
    skipped = []
    s_num = 0
    for i, f in enumerate(onlyfiles):
        doc = read_doc (f)
        if doc is None:
            skipped.append (f)
            s_num += 1
            continue
        word_in_doc = patt.split (doc[1])
        filtered_words = [word for word in set (word_in_doc) if word not in stopwords.words('russian')]
        len_of_file.append (len (word_in_doc))

        words.update (filtered_words)
#         for q in queries.iteritems ():
#             if q[1].issubset(filtered_words):
#                 docs_for_querie[q[0]].add (f)
        sys.stdout.write ("\r {}% scanned. {} skipped.                  ".format (round (i * 100. / num,3), s_num))
#         if i > 5:
#             break
    sys.stdout.write ("\r Done. {} skipped. {} Time Elapsed.            ".format (s_num, time.time () - t))
    avrg_len = np.mean (len_of_file)
    return words, skipped, docs_for_querie, avrg_len, len (onlyfiles) - len (skipped)

In [9]:
words, skipped, doc_f_q, avrg_len, num_of_docs = count_idf (queries)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

/usr/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /usr/lib64/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


 Done. 873 skipped. 20056.6933122 Time Elapsed.            

In [10]:
words.pop (u'')

16400

In [11]:
def update_idf (words, num_of_docs):
    idf = {}
    for d in words.iteritems ():
        res = np.log ((num_of_docs - d[1] + 0.5) / (d[1] + 0.5))
        if res > 0:
            idf[d[0]] = res
        else:
            idf[d[0]] = 0
    return idf

In [71]:
def get_bm25 (query, idf, doc, k, b, avrg_len):
    words = Counter (doc)
    bm25 = 0
    len_D = len (doc)
    for q in query:
        if q in idf:
            bm25 += idf[q] * idf[q] * (k + 1) / ( idf[q] + k * (1 - b + b*len_D / avrg_len))
    return bm25

In [72]:
def count_bm25 (queries, idf, avrg_len, b=0.75, k=2.0):
    mypath = 'content/'
    dirs = listdir(mypath)
    onlyfiles = []
    for d in dirs:
        onlyfiles.extend ([mypath + d + '/' + f for f in listdir(mypath + d) ])
    docs_for_querie = defaultdict (list)
    t = time.time ()
    num = len (onlyfiles)
    skipped = []
    s_num = 0
    for i, f in enumerate(onlyfiles):
        doc = read_doc (f)
        if doc is None:
            skipped.append (f)
            s_num += 1
            continue
        doc_name = doc[0]
        doc = patt.split (doc[1])
        filtered_words = [word for word in set (doc) if word not in stopwords.words('russian')]
        for q in queries.iteritems ():
            if len (q[1].intersection (filtered_words)) > 0:
                bm = get_bm25 (q[1], idf, doc,k,b,avrg_len)
                if len (docs_for_querie[q[0]]) == 10:
                    heapq.heappushpop (docs_for_querie[q[0]], (bm, doc_name))
                else:
                    heapq.heappush (docs_for_querie[q[0]], (bm, doc_name))
        sys.stdout.write ("\r {}% scanned. {} skipped.                  ".format (round (i * 100. / num,3), s_num))
#         if i > 5:
#             break
    sys.stdout.write ("\r Done. {} skipped. {} Time Elapsed.            ".format (s_num, time.time () - t))

    return docs_for_querie

In [14]:
idf = update_idf (words, num_of_docs)

In [73]:
docs = count_bm25 (queries, idf, avrg_len)

 Done. 873 skipped. 54822.9723499 Time Elapsed.            

In [74]:
urls_ids = {}
for u in urls.iteritems ():
    urls_ids[u[1] + '\n'] = u[0]

In [75]:
import pickle

In [76]:
pickle.dump (docs,open ("docs.txt",'w'))

In [77]:
def get_prediction (docs, urls_ids):
    pred = defaultdict (list)
    for q in docs.iteritems ():
        q[1].sort (key=lambda x: x[0],reverse=True)
        for do in q[1]:
            pred[q[0]].append (urls_ids[do[1]])
    return pred

In [78]:
pred = get_prediction (docs, urls_ids)

In [85]:
docs

defaultdict(list,
            {1: [(33.543952813032163,
               'fototerra.ru/Russia/Stanovoe/Borisovkirill-3380.html\n'),
              (33.529690815707234, 'kozhzdrav.ru/travmy/himicheskie-ozhogi\n'),
              (33.5174793494838,
               'pulse-mp3.ru/news/artik-asti-zachem-ya-tebe/2017-04-26-45898\n'),
              (33.512394801924742,
               'eplusik.ru/diabet/lechenie-diabeta-1-go-tipa-zolotym-usom\n'),
              (33.493092598299292,
               'otvetator.ru/dialog-kak-proyti-v-biblioteku-2-3-i-bolee\n'),
              (33.490047638841396,
               'almaty.satu.kz/Pult-na-vse-konditsionery.html\n'),
              (33.490047638841396,
               'bashtube.ru/video/7a020a2f490a4dc1930178b46f386361.html\n'),
              (33.489032819221329, 'otvet.mail.ru/profile/id14347556\n'),
              (33.485988860759122,
               'viahotel.ru/blog/category/polezno_znat/page-1\n'),
              (33.482945652594523, 'kazan.tiu.ru/Medilis-ts

In [79]:
res_file = open ("cub.csv", 'w')
res_file.write ('QueryId,DocumentId\n')

In [80]:
import random

In [81]:
for i in sorted (pred.iteritems ()):
    for d in i[1]:
        res_file.write ("{},{}\n".format (i[0],d))
    for tr in range (len (i[1]),10):
        res_file.write ("{},{}\n".format (i[0],random.randint (1,num_of_docs)))


In [82]:
res_file.close ()

In [83]:
!head cub.csv

QueryId,DocumentId
1,23236
1,1050
1,33511
1,30207
1,22446
1,16923
1,16843
1,34167
1,36782
